> First time use: follow instructions in the README.md file in this directory.

# **[PT]** Título


Para criar uma versão `html` sem o código fonte, fazer, na directoria "notebooks":

    jupyter nbconvert students_overview.ipynb --to html --no-input


# **[EN]** Title

_To create a `html` version without the code cells, do, in the "notebooks" directory_:

    jupyter nbconvert remissivas.ipynb --to html --no-input

## Setup

In [2]:
from timelinknb import current_time,current_machine, get_db
from ucalumni.config import default_db

db_spec = default_db
db = get_db(db_spec)
print(current_machine,current_time,f'db={db_spec}')

imac-jrc.local 2022-05-18 17:44:12.740429 db=('sqlite', 'fauc.db')


## **[PT]** Naturalidades

In [3]:
from timelinknb.pandas import attribute_values

naturalidades = attribute_values('naturalidade')

In [4]:
naturalidades[naturalidades['count']<3].sort_values('count', ascending=False).head(40)

,count,date_in,date_max
value,,,
Abadia,2,1836-10-14,1886-10-14
"Santa Eulália, Nespereira",2,1745-10-01,1766-10-01
Santa Eulália de Constância,2,1849-10-31,1851-10-09
Santa Eulália da Serra,2,1777-10-01,1813-10-29
"Santa Eugénia, Vila Real",2,1815-10-30,1880-10-15
Santa Cruz do Bouro,2,1876-10-12,1876-10-12
Santa Cruz de Lumiares,2,0000-00-00,1734-01-27
"Santa Comba, Vila Real",2,1817-10-06,1842-10-17
"Santa Comba, Fornelos",2,1750-10-01,1819-10-30


## Usar Toponímia das cartas militares

In [1]:
import pandas as pd
toponimia = pd.read_csv('../extras/toponimia/toponima-militar.csv')
toponimia['nome_concelho'] = toponimia['nome']+", "+toponimia['concelho']
toponimia.head()

FileNotFoundError: [Errno 2] No such file or directory: '../extras/toponimia/toponima-militar.csv'

In [27]:
toponimia[toponimia.nome == "Lisboa"]

,nome,codigoine,xx,yy,cartamilitar,tipo,concelho,distrito,obs,id
64490,Lisboa,110624,112483,194793,431,SC,Lisboa,Lisboa,(null),64491


In [36]:
for local in naturalidades.index.values[:100]:
    clocal = local.lstrip("?-/").rstrip("?-/").strip()
    topo_list = toponimia.loc[toponimia.nome == local].index.values
    if len(topo_list) == 0:
        print(f"|{clocal}| not found")
    else:
        for t in topo_list:
            lower(toponimo) = lower(toponimia.loc[[t]].iloc[0])
            print(clocal,toponimo['nome'],toponimo['concelho'],toponimo['distrito'])

|"""
                mostrou que seu pai se chamava Manuel Ribeiro e por ordem do mesmo senhor se lhe passaram as cartas com o nome de seu pai Manuel Ribeiro e fiz esta declaração para que não faça dúvida o assento das matrículas
            """| not found
|(Estado de Minas Gerais)Brasil| not found
|Lisboa| not found
|Vila Franca| not found
|Lisboa| not found
|23031686| not found
|28111637| not found
|| not found
|do Campo| not found
|Inglesa| not found
|Lisboa| not found
|a| not found
A de Barros A de Barros Sernancelhe Viseu
|A dos Francos| not found
|a- Velha| not found
|Abaças| not found
Abade Abade Chamusca Santarém
|Abade de São Romão| not found
Abadia Abadia Amares Braga
Abadia Abadia Covilhã Castelo Branco
Abadia Abadia Leiria Leiria
Abadia Abadia Mafra Lisboa
Abadia Abadia Mangualde Viseu
Abadia Abadia Mêda Guarda
Abadia Abadia Miranda do Douro Bragança
Abadia Abadia Ourém Santarém
Abadia Abadia Pinhel Guarda
Abadia Abadia Sobral de Monte Agraço Lisboa
Abadia Abadia Tomar Sant

## **[PT]** Contagens